In [1]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
from sentence_transformers import SentenceTransformer
documents = fetch_20newsgroups(subset='all')['data']

In [2]:
topic_model = BERTopic()
documents = pd.DataFrame({"Document": documents,
                      "ID": range(len(documents)),
                      "Topic": None})

In [3]:
%%time
#Extract embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(
    documents.Document,
    show_progress_bar=False
)

CPU times: user 3min 8s, sys: 9.73 s, total: 3min 18s
Wall time: 35.6 s


In [4]:
%%time
# Dimensionality Reduction
umap_embeddings = topic_model._reduce_dimensionality(embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
CPU times: user 9min 35s, sys: 8.98 s, total: 9min 44s
Wall time: 21.9 s


In [7]:
%%time
# Cluster UMAP embeddings with HDBSCAN
documents, probabilities = topic_model._cluster_embeddings(umap_embeddings, documents)

CPU times: user 801 ms, sys: 20 ms, total: 821 ms
Wall time: 886 ms


In [8]:
%%time
# Sort and Map Topic IDs by their frequency
if not topic_model.nr_topics:
    documents = topic_model._sort_mappings_by_frequency(documents)

# Extract topics by calculating c-TF-IDF
topic_model._extract_topics(documents) # does both topic extraction and representation

CPU times: user 6.89 s, sys: 152 ms, total: 7.04 s
Wall time: 7.04 s
